<h1>Project Data Mining</h1>
<br><h3>Sentiment Analysis on tweets</h3>
<br><b>Editors: </b>
<br>Aslanidis Theofanis - Athinaios Konstantinos

<b>1)</b> Importing all needed packages: <br>
<ul>
    <li><b>nltk:</b> used for tokenization
    <li><b>pandas:</b> data representation
<ul>


In [35]:
import numpy as np
import pandas as pd
import matplotlib as plt
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize

<b>2)</b> Loading tweeter datasets for training and testing:
<ul>
    <li><u>Value</u> for negative-positive-neutral
    <li><u>Tweet</u> for the text
</ul>

In [36]:
train_data = pd.read_csv("../twitter_data/train2017.tsv", names=['Value', 'Tweet'], sep='\t', usecols = [2, 3])
test_data = pd.read_csv("../twitter_data/test2017.tsv", names=['Value', 'Tweet'], sep='\t', usecols = [2, 3])
train.head()

,Value,Tweet
0,positive,Gas by my house hit $3.39!!!! I'm going to Cha...
1,negative,"Theo Walcott is still shit, watch Rafa and Joh..."
2,negative,"its not that I'm a GSP fan, i just hate Nick D..."
3,negative,Iranian general says Israel's Iron Dome can't ...
4,neutral,"Tehran, Mon Amour: Obama Tried to Establish Ti..."


<h4>Our pipeline:</h4><br>
<ol>
    <li>Load the raw text.
    <li>Split into tokens.
    <li>Convert to lowercase.
    <li>Remove punctuation from each token.
    <li>Filter out remaining tokens that are not alphabetic.
    <li>Filter out tokens that are stop words.
<ol>


In [42]:
train_data[1] = word_tokenize(train_data[1])

KeyError: 1